In [1]:
training_corpus = [
    "low", "lower", "lowest",
    "new", "newer", "newest",
    "wide", "wider", "widest",
    "slow", "slower", "slowest",
    "bright", "brighter", "brightest",
    "smart", "smarter", "smartest",
    "quick", "quicker", "quickest",
    "cold", "colder", "coldest",
    "strong", "stronger", "strongest"
]


Each word is split into characters and marked with an end symbol \</w>


In [2]:
from collections import Counter, defaultdict

def build_vocab(corpus):
    vocab = Counter()
    for word in corpus:
        vocab[" ".join(list(word)) + " </w>"] += 1
    return vocab


Finds most frequent adjacent characters/subwords <br>
These pairs are candidates for merging

In [3]:
def get_pair_frequencies(vocab):
    pairs = defaultdict(int)
    for word, freq in vocab.items():
        symbols = word.split()
        for i in range(len(symbols) - 1):
            pairs[(symbols[i], symbols[i+1])] += freq
    return pairs


Merge the Most Frequent Pair

In [4]:
def merge_pair(pair, vocab):
    new_vocab = {}
    bigram = " ".join(pair)
    merged = "".join(pair)

    for word in vocab:
        new_word = word.replace(bigram, merged)
        new_vocab[new_word] = vocab[word]

    return new_vocab


Train the BPE Model

In [5]:
def train_bpe(corpus, num_merges):
    vocab = build_vocab(corpus)
    merges = []

    for _ in range(num_merges):
        pairs = get_pair_frequencies(vocab)
        if not pairs:
            break

        best_pair = max(pairs, key=pairs.get)
        merges.append(best_pair)
        vocab = merge_pair(best_pair, vocab)

    return merges


train with multiple merges

In [6]:
bpe_merges = train_bpe(training_corpus, num_merges=30)

print("Learned BPE merges:")
for m in bpe_merges:
    print(m)


Learned BPE merges:
('s', 't')
('e', 'r')
('er', '</w>')
('e', 'st')
('est', '</w>')
('l', 'o')
('lo', 'w')
('n', 'e')
('ne', 'w')
('w', 'i')
('wi', 'd')
('s', 'low')
('b', 'r')
('br', 'i')
('bri', 'g')
('brig', 'h')
('brigh', 't')
('s', 'm')
('sm', 'a')
('sma', 'r')
('smar', 't')
('q', 'u')
('qu', 'i')
('qui', 'c')
('quic', 'k')
('c', 'o')
('co', 'l')
('col', 'd')
('st', 'r')
('str', 'o')


In [7]:
test_corpus = [
    "smartest",
    "quickest",
    "slowest",
    "newer",
    "stronger"
]


Apply BPE Tokenization on Test Words

In [8]:
def bpe_tokenize(word, merges):
    tokens = list(word) + ["</w>"]

    for merge in merges:
        i = 0
        while i < len(tokens) - 1:
            if tokens[i] == merge[0] and tokens[i+1] == merge[1]:
                tokens[i:i+2] = ["".join(merge)]
            else:
                i += 1

    return tokens


Tokenize and Display Output

In [9]:
print("\nTokenized Test Corpus:")
for word in test_corpus:
    print(word, "→", bpe_tokenize(word, bpe_merges))



Tokenized Test Corpus:
smartest → ['smart', 'est</w>']
quickest → ['quick', 'est</w>']
slowest → ['slow', 'est</w>']
newer → ['new', 'er</w>']
stronger → ['stro', 'n', 'g', 'er</w>']
